<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       BinaryMatrixOp Function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>

<p style = 'font-size:20px;font-family:Arial'><b>BinaryMatrixOp</b></p>
<p style = 'font-size:16px;font-family:Arial'>The BinaryMatrixOp() function performs a point-wise mathematical operation on two matrices with an equal number of wavelets and for which corresponding wavelets have an equal number of data points. A point-wise operation signifies that a mathematical operation is performed on one sample point at a time. Allowed mathematical operations are addition, subtraction, multiplication and division.</p>


<p style = 'font-size:16px;font-family:Arial'>The common uses of BinaryMatrixOp() function are:</p>
<li style = 'font-size:16px;font-family:Arial'><code>Addition:</code> Restore trends to a time matrix before using the model for forecasting.</li>
<li style = 'font-size:16px;font-family:Arial'><code>Subtraction:</code> Subtract trends from a time matrix to create a model from it.</li>
<li style = 'font-size:16px;font-family:Arial'><code>Multiplication:</code> Apply a low pass, band pass, or high pass filter to a time matrix.</li>
<li style = 'font-size:16px;font-family:Arial'><code>Division:</code> Divide the primary matrix by the secondary matrix.</li>
</p>
 


<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDMatrix,
    BinaryMatrixOp,
    db_drop_view,
    remove_context,
    copy_to_sql,
    db_drop_table
    )

from teradatasqlalchemy.types import *

from teradataml import to_numeric
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_BinaryMatrixOp_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we will get the time series data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("uaf", ["binary_matrix_complex_left", "binary_matrix_complex_right"])
load_example_data("uaf", ["binary_matrix_real_left", "binary_matrix_real_right"])

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
data1 = DataFrame.from_table("binary_matrix_complex_left")
data2 = DataFrame.from_table("binary_matrix_complex_right")
data3 = DataFrame.from_table("binary_matrix_real_left")
data4 = DataFrame.from_table("binary_matrix_real_right")

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. BinaryMatrixOp</b>
<p style = 'font-size:16px;font-family:Arial'>The BinaryMatrixOp() function takes two equally sized logical-runtime matrices as input - the first matrix referenced in the function call ("data1") is a primary matrix, whereas the second matrix referenced ("data2") is a secondary matrix. The significance of being a primary or secondary matrix in each of the allowed mathematical operation is as follows:</p>
<li style = 'font-size:16px;font-family:Arial'><code>SUB:</code> The secondary matrix is subtracted from the primary matrix.</li>
<li style = 'font-size:16px;font-family:Arial'><code>ADD:</code> The secondary matrix is added to the primary matrix.</li>
<li style = 'font-size:16px;font-family:Arial'><code>MUL:</code> The primary matrix is multiplied by the secondary matrix.</li>
<li style = 'font-size:16px;font-family:Arial'><code>DIV:</code> The primary matrix is divided by the secondary matrix.</li></p>

<p style = 'font-size:16px;font-family:Arial'>Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(BinaryMatrixOp)

<p style = 'font-size:16px;font-family:Arial'>The first step is to convert the data into TDMatrix, which is required for the input time series which are passed to the BinaryMatrixOp function.</p>

In [ ]:
binary_matrix_complex_left = TDMatrix(data=data1,
                                          id="id",
                                          row_index="seq",
                                          column_index="tick",
                                          row_index_style="SEQUENCE",
                                          column_index_style="SEQUENCE",
                                          payload_field=["real_val", "imaginary_val"],
                                          payload_content="COMPLEX")

In [ ]:
binary_matrix_complex_right = TDMatrix(data=data2,
                                           id="id",
                                           row_index="seq",
                                           column_index="tick",
                                           row_index_style="SEQUENCE",
                                           column_index_style="SEQUENCE",
                                           payload_field=["real_val", "imaginary_val"],
                                           payload_content="COMPLEX")

<p style = 'font-size:16px;font-family:Arial'>We use the BinaryMatrixOp to perform addition of two time matrix of equal size.</p>

In [ ]:
uaf_out = BinaryMatrixOp(data1=binary_matrix_complex_left,
                               data2=binary_matrix_complex_right,
                               data2_filter_expr=binary_matrix_complex_right.id==1,
                               math_op="ADD",
                               input_fmt_input_mode="MANY2ONE")

binarymatrix_df = uaf_out.result
binarymatrix_df

<p style = 'font-size:16px;font-family:Arial'>We use the BinaryMatrixOp to perform multiplication operation in MATCH mode between two matrices holding REAL payload.</p>

In [ ]:
binary_matrix_real_left = TDMatrix(data=data3,
                                       id="id",
                                       row_index="seq",
                                       column_index="tick",
                                       row_index_style="SEQUENCE",
                                       column_index_style="SEQUENCE",
                                       payload_field="a",
                                       payload_content="REAL") 

In [ ]:
binary_matrix_real_right = TDMatrix(data=data4,
                                        id="id",
                                        row_index="seq",
                                        column_index="tick",
                                        row_index_style="SEQUENCE",
                                        column_index_style="SEQUENCE",
                                        payload_field="b",
                                        payload_content="REAL")

In [ ]:
uaf_out_2 = BinaryMatrixOp(data1=binary_matrix_real_left,
                               data2=binary_matrix_real_right,
                               math_op="MUL",
                               input_fmt_input_mode="MATCH")

binarymatrix_mul_df = uaf_out_2.result
binarymatrix_mul_df

<p style = 'font-size:16px;font-family:Arial'>The result matrix produced by the BinaryMatrixOp() function is of the same size, 'N * M', as of its inputs and inherits the TDMatrix identifier (id), from the primary matrix.</p>


<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables created above.</p>

In [ ]:
db_drop_table("binary_matrix_complex_left")
db_drop_table("binary_matrix_complex_right")
db_drop_table("binary_matrix_real_left")
db_drop_table("binary_matrix_real_right")

In [ ]:
remove_context()

<hr style="height:1px;border:none;">

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>BinaryMatrixOp function reference: <a href = 'https://docs.teradata.com/search/all?query=BinaryMatrixOp&content-lang=en-US'>here</a></li>
    
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>